# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/4a32fd7bf343239da3cd0ecc0a5f8cad263ae0415894ae7fc96542f2.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Pine Bush, New York, United States**, and the stations the data comes from are shown on the map below.

In [2]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'4a32fd7bf343239da3cd0ecc0a5f8cad263ae0415894ae7fc96542f2')

FileNotFoundError: File b'data/C2A2_data/BinSize_d400.csv' does not exist

## 1. Graph Max and Min Rain Data from 2005 - 2014

In [3]:
# get the data from the csv file and create DataFrame object with parsed dates.
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/4a32fd7bf343239da3cd0ecc0a5f8cad263ae0415894ae7fc96542f2.csv', 
                 parse_dates=[1])

# convert 'Date' columns to pandas datetime objects
pd.to_datetime(df['Date'])

FileNotFoundError: File b'data/C2A2_data/BinnedCsvs_d400/4a32fd7bf343239da3cd0ecc0a5f8cad263ae0415894ae7fc96542f2.csv' does not exist

In [ ]:
# select data by specific datetime attributes
df = (df.where(
    df['Date'].dt.year.isin(range(2005, 2015))  # get years 2005-2014
        & (df['Date'].dt.month!=2) 
        & (df['Date'].dt.day!=29))              # and exclude leap days
    .dropna()
    .sort_values('Date'))

df['Day of Year'] = df['Date'].dayofyear
df.head()

In [ ]:
days_df = df.copy()

# groupby ordinal day of the year. 
days_df = days_df.groupby('Day of Year')['Data_Value'].agg({'Record High': np.max, 'Record Low': np.min})

days_df.shape # 337 ...?

# what's missing?
missing_days = list()
for day in range(1, 367):
    if day not in days_df.index:
        missing_days.append(day)

# check if these exist in original 'Day of Year' before groupby
df.where(df['Day of Year'].isin(missing_days)).dropna()  # nada.


plt.figure()
plt.plot(days_df, '-o')

In [ ]:
df['Day'] = df['Date'].map(lambda x: '{:02}-{:02}'.format(x.month, x.day))

In [ ]:
import numpy as np
records = df.groupby('Day')['Data_Value'].agg({'Record High': np.max, 'Record Low': np.min})
records.sort_index()

In [ ]:
records.shape